In [1]:
import os
import tifffile
import numpy as np
from src.utils.utils_dataprocessing import get_all_files, split_and_save_tensor, normalize, resample, fourier_interpolate
from src.SN2N_2D.constants_2d import rawdataFolder, datasetsFolder, resultFolder


raw_map_list = get_all_files(rawdataFolder)
print(raw_map_list[0])
raw_map = np.asarray(tifffile.imread(raw_map_list[0]))
normalized_map = normalize(raw_map, mode='2d')
tifffile.imwrite(f'{resultFolder}/normalized_map.tif', normalized_map) 


/home/tyche/paddle_SN2N/data/data_2d/raw_data/1.tif


In [2]:
datasets = get_all_files(datasetsFolder)
print(datasets[0])
chunk = np.load(datasets[100])['arr_0']
print(chunk.shape)
tifffile.imwrite(f'{resultFolder}/chunk.tif', chunk)

/home/tyche/paddle_SN2N/data/data_2d/datasets/0_128_128_128.npz
(128, 128)


In [3]:
import torch
import numpy as np
import tifffile

kernel = torch.tensor([[[1, 0], [0, 1]],
                      [[0, 1], [1, 0]]]).float() / 2

# 使用vstack进行垂直堆叠
vertical_kernel = np.vstack((kernel[0], kernel[1]))

# 保存为TIF文件
tifffile.imwrite(f'{resultFolder}/kernel_vertical.tif', vertical_kernel)

In [4]:
import torch
from src.utils.utils_dataprocessing import fourier_interpolate

kernel = torch.tensor([[[1, 0], [0, 1]],
                      [[0, 1], [1, 0]]]).float() / 2
out_channels, *spatial_dims = kernel.shape
kernel = kernel.view(out_channels, 1, *spatial_dims)
conv_layer = torch.nn.Conv2d(in_channels=1, out_channels=2, kernel_size=2, stride=2, padding=0, bias=False)
conv_layer.weight.data = kernel
chunk = torch.from_numpy(np.load(datasets[100])['arr_0'])
print(chunk.shape)
chunk = conv_layer(chunk.view(1, *chunk.shape))
print(chunk.shape)
vertical_chunk = np.vstack((chunk[0].detach().numpy(), np.zeros((1, 128), dtype=np.float32), chunk[1].detach().numpy()))
tifffile.imwrite(f'{resultFolder}/down_sampled.tif', vertical_chunk)
chunk = fourier_interpolate(chunk.view(2, 1, 128, 128)) * 16
print(chunk.shape)
vertical_chunk1 = np.vstack((chunk[0][0].detach().numpy(), np.zeros((1, 256), dtype=np.float32), chunk[1][0].detach().numpy()))
tifffile.imwrite(f'{resultFolder}/up_sampled.tif', vertical_chunk1)

torch.Size([128, 128])
torch.Size([2, 64, 64])


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 64 and the array at index 1 has size 128

In [ ]:
from src.utils.utils_train_predict import try_all_gpus
from src.SN2N_2D.constants_2d import model, paramsFolder
devices = try_all_gpus()
model = model.to(devices[0])
model = torch.nn.DataParallel(model, device_ids=[0])
def init_weights(m):
    if type(m) == torch.nn.Linear or type(m) == torch.nn.Conv3d or type(m) == torch.nn.Conv2d:  
        torch.nn.init.xavier_uniform_(m.weight)
current_epoch = len(get_all_files(paramsFolder))
print(current_epoch)
if current_epoch != 0:
    state_dict = torch.load(f'{paramsFolder}/checkPoint_{current_epoch - 1}')
    missing_keys, unexpected_keys = model.load_state_dict(state_dict)
    if missing_keys:
        print(f"missing_keys: {missing_keys}")
    if unexpected_keys:
        print(f"unused_keys: {unexpected_keys}")
    
    print(f'load {paramsFolder}/checkPoint_{current_epoch - 1}')
else:
    model.apply(init_weights)
    print(f'no params found, randomly init model')
total_params = sum(p.numel() for p in model.parameters())
print(f"num_parameters: {total_params}")
chunk = chunk.to(device=devices[0])
chunk = model(chunk)
vertical_chunk1 = np.vstack((chunk[0][0].cpu().detach().numpy(), np.zeros((1, 256), dtype=np.float32), chunk[1][0].cpu().detach().numpy()))
tifffile.imwrite(f'{resultFolder}/predicted.tif', vertical_chunk1)

2
load /home/tyche/paddle_SN2N/data/data_2d/params/checkPoint_1
num_parameters: 6494716
